In [1]:
import json

In [2]:
jsonData = """
{
"id": 24717,
"cuisine": "indian",
"ingredients": [
"tumeric",
"vegetable stock",
"tomatoes",
"garam masala",
"naan",
"red lentils",
"red chili peppers",
"onions",
"spinach",
"sweet potatoes"
]
}
"""

In [3]:
jsonData = """
{
    "researcher": {
        "name": "Ford Prefect",
        "species": "Betelgeusian",
        "relatives": [
            {
                "name": "Zaphod Beeblebrox",
                "species": "Betelgeusian"
            }
        ]
    }
}
"""

In [4]:
jsonToPython = json.loads(jsonData)

print(jsonToPython)


{'researcher': {'relatives': [{'name': 'Zaphod Beeblebrox', 'species': 'Betelgeusian'}], 'name': 'Ford Prefect', 'species': 'Betelgeusian'}}


In [5]:
jsonToPython['researcher']

{'name': 'Ford Prefect',
 'relatives': [{'name': 'Zaphod Beeblebrox', 'species': 'Betelgeusian'}],
 'species': 'Betelgeusian'}

In [6]:
with open("train.json", 'r') as f:
    datastore = json.load(f)

In [7]:
print(datastore[1])

{'id': 25693, 'cuisine': 'southern_us', 'ingredients': ['plain flour', 'ground pepper', 'salt', 'tomatoes', 'ground black pepper', 'thyme', 'eggs', 'green tomatoes', 'yellow corn meal', 'milk', 'vegetable oil']}


In [8]:
ingredients = []
most_ingredients = 0
for dish in datastore:
    count_ingredients = 0
    for ingredient in dish["ingredients"]:
        ingredients.append(ingredient)
        count_ingredients += 1
    if count_ingredients > most_ingredients:
        most_ingredients = count_ingredients
ingredients = set(ingredients)        
print(len(ingredients))

6714


In [9]:
print(most_ingredients)

65


In [10]:
ingredient_dict = {}
ing_reverse_dict = {}
ingredient_dict["blank"]=0
ing_reverse_dict[0]="blank"
for index,ingredient in enumerate(ingredients):
    ingredient_dict[ingredient] = index+1
    ing_reverse_dict[index+1] = ingredient

In [11]:
print(len(ingredient_dict))

6715


In [12]:
#create training sets
training_ingredients = []
for dish in datastore:
    dish_in_index = []
    for ingredient in dish["ingredients"]:
        dish_in_index.append(ingredient_dict[ingredient])
    for i in range(len(dish_in_index),most_ingredients+1):
        dish_in_index.append(0)
    training_ingredients.append(dish_in_index)
print(len(training_ingredients))

39774


In [13]:
cuisines = []
for dish in datastore:
    cuisines.append(dish["cuisine"])
cuisines = set(cuisines)
cuisine_dict = {}
cui_reverse_dict = {}
for index,cuisine in enumerate(cuisines):
    cuisine_dict[cuisine] = index
    cui_reverse_dict[index] = cuisine

In [14]:
print(cuisine_dict)

{'cajun_creole': 0, 'thai': 1, 'jamaican': 6, 'southern_us': 3, 'filipino': 18, 'vietnamese': 7, 'greek': 8, 'mexican': 9, 'korean': 10, 'british': 15, 'french': 11, 'chinese': 12, 'spanish': 13, 'japanese': 14, 'moroccan': 5, 'irish': 16, 'italian': 17, 'russian': 2, 'indian': 19, 'brazilian': 4}


In [15]:
#one-hot coding labels
labels = []
for dish in datastore:
    one_hot_cuisine = []
    for i in range(len(cuisines)):
        if i == cuisine_dict[dish["cuisine"]]:
            one_hot_cuisine.append(1)
        else:
            one_hot_cuisine.append(0)
    labels.append(one_hot_cuisine)

In [16]:
#logits_to_cuisine = cui_reverse_dict[np.argmax(logits, 1)]

In [17]:
#imports
import collections
import sys
import numpy
import numpy as np
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras.layers import Lambda
from keras.layers.embeddings import Embedding
from keras.layers import Flatten

Using TensorFlow backend.


In [18]:
#turn into arrays
labels = np.asarray(labels)
training_ingredients = np.asarray(training_ingredients)
print(labels.shape)
print(training_ingredients.shape)

(39774, 20)
(39774, 66)


In [19]:
model = Sequential()
model.add(Embedding(len(ingredient_dict), 64, input_length=66))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation=None))
model.add(Lambda(lambda x: K.tf.nn.softmax(x)))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(training_ingredients, labels, batch_size=200, epochs=10, validation_split=0.1)

Train on 35796 samples, validate on 3978 samples
Epoch 1/10
35796/35796 [==============================] - 20s 553us/step - loss: 1.9432 - acc: 0.4324 - val_loss: 1.2680 - val_acc: 0.6242

In [20]:
print(training_ingredients[1])
print(training_ingredients[1].shape)

[4291 1296 2595 5499  580 1496  610 6530 5843 5080  876    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
(66,)


In [25]:
test_dish = training_ingredients[1].reshape(-1,66)
logits=model.predict(test_dish)
logits_to_cuisine = cui_reverse_dict[np.argmax(logits)]

In [29]:
print(logits_to_cuisine)

southern_us


In [27]:
print(len(cui_reverse_dict))

20


In [30]:
with open("test.json", 'r') as f:
    datastore = json.load(f)

In [32]:
#create training sets
test_ingredients = []
for dish in datastore:
    dish_in_index = []
    for ingredient in dish["ingredients"]:
        if ingredient in ingredient_dict.keys():
            dish_in_index.append(ingredient_dict[ingredient])
        else:
            dish_in_index.append(0)
    for i in range(len(dish_in_index),most_ingredients+1):
        dish_in_index.append(0)
    test_ingredients.append(dish_in_index)
print(len(test_ingredients))

9944


In [35]:
test_ingredients = np.asarray(test_ingredients)

In [36]:
logits=model.predict(test_ingredients)

In [37]:
import csv
with open('names.csv', 'w') as csvfile:
    fieldnames = ['id', 'cuisine']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for dish,logit in zip(datastore,logits):
        logits_to_cuisine = cui_reverse_dict[np.argmax(logit)]
        writer.writerow({'id': dish["id"], 'cuisine': logits_to_cuisine})